In [37]:
#!/bin/bash -e

# Use of ANNIF library

Ce notebook contient toutes les étapes de l'utilisation de la librairie annif (doc d'installation de toutes les librairies à prévoir).
- Formatage des données pour utilisation dans ANNIF
- Entrainement d'un modèle 
- Utilisation de pipelines pour tester plusieurs modèles
- Recherche des meilleurs paramètres  

## Setup 

### Packages

In [1]:
# Import librairies
import os
import csv
import pandas as pd

### Graphical parameters

### Paths

In [2]:
# Create folders if needed
list_folder = [
    "ANNIF", 
    "ANNIF/data", "ANNIF/reports",
    "ANNIF/data/train", "ANNIF/data/test"]

for folder in list_folder:
    if not os.path.exists(folder):
        os.makedirs(folder)
    else:
        print(f"Folder {folder} already exists")



Folder ANNIF already exists
Folder ANNIF/data already exists
Folder ANNIF/reports already exists
Folder ANNIF/data/train already exists
Folder ANNIF/data/test already exists


In [3]:
# Set current directory
annif_path = os.getcwd() + "/ANNIF"
os.chdir(annif_path)

In [4]:
# Set paths
data_path = "./../data"
annif_data_path = annif_path + "/data"
annif_report_path = annif_path + "/reports"

### Files

In [5]:
# Select data to use
data = "working_data_sans_dewey.pkl"
rameau_file = "extraction/liste_concepts_rameau_sans_vedetteConstruites.csv"
rameau_file_with_vedetteConstruites = "extraction/liste_concepts_rameau.csv" 

In [115]:
# Merge predictions to existing predictions?
save_predictions = True

## Create datasets

### Import data

In [6]:
# Import working data
df = pd.read_pickle(os.path.join(data_path, data))
print(df.shape)

(154508, 9)


In [7]:
df.head()

,PPN,TITRE,RESUME,RAMEAU,DEWEY,DESCR,presence_chaine_indexation,rameau_chaines_index,rameau_concepts
0,000002364,La culture pour vivre,Mort de la culture populaire en France. Mutati...,Culture populaire;Diffusion de la culture;Poli...,840,La culture pour vivre Mort de la culture popul...,False,"[Culture populaire, Diffusion de la culture, P...","[Culture populaire, Diffusion de la culture, P..."
1,000014877,"La nuit, le jour : essai psychanalytique sur l...","Discontinuité, latence, rétablissement d’une c...",Complexe de castration;Psychanalyse;Rêves,154.63,"La nuit, le jour : essai psychanalytique sur l...",False,"[Complexe de castration, Psychanalyse, Rêves]","[Complexe de castration, Psychanalyse, Rêves]"
2,000021857,"Ruptures, cultures","Il faut imaginer Robinson sur son île, au mome...",Culture,840,"Ruptures, cultures Il faut imaginer Robinson s...",False,[Culture],[Culture]
3,00002564X,La révolution structurale,"Mutations ou crises, les brusques accès de fiè...",Structuralisme,100,"La révolution structurale Mutations ou crises,...",False,[Structuralisme],[Structuralisme]
4,000026352,La Destruction du temple,"Oswald tire sur Kennedy. Jusque-là, c'est bon,...",Science-fiction américaine -- Traductions fran...,830,La Destruction du temple Oswald tire sur Kenne...,True,[Science-fiction américaine -- Traductions fra...,"[Science-fiction américaine, Traductions franç..."


In [8]:
# Import file of RAMEAU concepts
rameau = pd.read_csv(os.path.join(data_path, rameau_file), encoding="latin-1")
print(rameau.shape)
rameau.head(20)

(85876, 2)


,PPN,NOM
0,157992527,Kirp?n
1,110140494,Militaires artistes
2,028492161,Militaires romains
3,028521757,Militaires prussiens
4,029895561,Sa-skya-pa
5,031875459,Militaires réunionnais
6,032370083,Construction à l'épreuve de la sécheresse
7,032878117,Missionnaires suisses
8,034423982,Militaires ivoiriens
9,034686940,Outils à métaux


### Create vocabulary file

In [9]:
# Associate concepts with URI
BASE_URI = 'https://www.idref.fr/'
rameau["URI"] = rameau["PPN"].apply(lambda x: BASE_URI + x)
rameau.head(20)

,PPN,NOM,URI
0,157992527,Kirp?n,https://www.idref.fr/157992527
1,110140494,Militaires artistes,https://www.idref.fr/110140494
2,028492161,Militaires romains,https://www.idref.fr/028492161
3,028521757,Militaires prussiens,https://www.idref.fr/028521757
4,029895561,Sa-skya-pa,https://www.idref.fr/029895561
5,031875459,Militaires réunionnais,https://www.idref.fr/031875459
6,032370083,Construction à l'épreuve de la sécheresse,https://www.idref.fr/032370083
7,032878117,Missionnaires suisses,https://www.idref.fr/032878117
8,034423982,Militaires ivoiriens,https://www.idref.fr/034423982
9,034686940,Outils à métaux,https://www.idref.fr/034686940


In [10]:
# Create dictionnary of URI
label2uri = {k:v for k,v in zip(rameau["NOM"], rameau["URI"].astype(str))}

In [11]:
# Save URI to csv file
rameau.to_csv(os.path.join(data_path, str(rameau_file[:-4] + '_withURI.csv')))

In [12]:
# Create vocabulary file
vocab_filename = os.path.join(annif_data_path,'subjects.csv')
vocab = pd.DataFrame(rameau[["NOM", "URI"]])
vocab.columns = ["label_fr", "uri"]
vocab.to_csv(vocab_filename, encoding='utf-8', index=None)
vocab.head(10)

,label_fr,uri
0,Kirp?n,https://www.idref.fr/157992527
1,Militaires artistes,https://www.idref.fr/110140494
2,Militaires romains,https://www.idref.fr/028492161
3,Militaires prussiens,https://www.idref.fr/028521757
4,Sa-skya-pa,https://www.idref.fr/029895561
5,Militaires réunionnais,https://www.idref.fr/031875459
6,Construction à l'épreuve de la sécheresse,https://www.idref.fr/032370083
7,Missionnaires suisses,https://www.idref.fr/032878117
8,Militaires ivoiriens,https://www.idref.fr/034423982
9,Outils à métaux,https://www.idref.fr/034686940


In [13]:
# Check import with bash:
! head {vocab_filename}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
label_fr,uri
Kirp?n,https://www.idref.fr/157992527
Militaires artistes,https://www.idref.fr/110140494
Militaires romains,https://www.idref.fr/028492161
Militaires prussiens,https://www.idref.fr/028521757
Sa-skya-pa,https://www.idref.fr/029895561
Militaires réunionnais,https://www.idref.fr/031875459
Construction à l'épreuve de la sécheresse,https://www.idref.fr/032370083
Missionnaires suisses,https://www.idref.fr/032878117
Militaires ivoiriens,https://www.idref.fr/034423982


In [14]:
# Check number of concepts in the vocabulary file:
! wc -l < {vocab_filename}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
85877


In [15]:
# Load vocabulary file in ANNIF
! annif load-vocab rameau {vocab_filename}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Loading vocabulary from CSV file /home/aurelie/ABES/labo-indexation-ai/ANNIF/data/subjects.csv...
updating existing subject index
saving vocabulary into SKOS file data/vocabs/rameau/subjects.ttl


### Datasets for training and evaluation

#### Define train and test sets (using split performed by Jean Luc Prieto)

In [16]:
# Define train and test sets 
train_set_ppn = pd.read_excel(os.path.join(data_path, "training_data.xlsx"), dtype=str)
print("dimension of train set: ", train_set_ppn.shape)
test_set_ppn = pd.read_excel(os.path.join(data_path, "testing_data.xlsx"), dtype=str)
print("dimension of test set: ", test_set_ppn.shape)

dimension of train set:  (125264, 1)
dimension of test set:  (29244, 1)


In [17]:
df_train = pd.merge(df, train_set_ppn, on="PPN", how="inner")
print("dimension of train set: ", df_train.shape)
df_test = pd.merge(df, test_set_ppn, on="PPN", how="inner")
print("dimension of test set: ", df_test.shape)

dimension of train set:  (115853, 9)
dimension of test set:  (27070, 9)


#### Create TSV files (short-text-document)
see: https://github.com/NatLibFi/Annif/wiki/Document-corpus-formats#short-text-document-corpus-tsv-file

In [18]:
# Format URIS for ANNIF
def format_for_annif(labels):
    uris = []
    for label in labels:
        if label in label2uri.keys():
            uris.append(label2uri[label])
    uris_formated = " ".join(uris)
    return uris_formated

In [19]:
# Apply on train data
df_train["rameau_concept_formatted_for_annif"] = df_train["rameau_concepts"].apply(lambda x: format_for_annif(x))
df_train["DESCR_cleaned"] = df_train["DESCR"].apply(lambda x: " ".join(x.split()))
df_train[["DESCR_cleaned", "rameau_concept_formatted_for_annif"]].head()

,DESCR_cleaned,rameau_concept_formatted_for_annif
0,Une politique pour La Réunion Michel Debré a é...,https://www.idref.fr/027274292 https://www.idr...
1,L'Aparole électorale La 4ème de couv. indique ...,https://www.idref.fr/027225763
2,La face cachée du soleil : énergie solaire et ...,https://www.idref.fr/027391027 https://www.idr...
3,La voie et sa vertu : Tao-tê-king La 4e de cou...,https://www.idref.fr/027847896 https://www.idr...
4,Akavak Akavak n'a pas encore quatorze ans. Et ...,https://www.idref.fr/027235262


In [20]:
# Save TSV file for ANNIF
train_tsv_path = os.path.join(annif_data_path, "rameau-train.tsv")
with open(train_tsv_path, 'w', encoding='utf-8') as output_train_file:
        for index, row in df_train.iterrows():
                print(row['DESCR_cleaned'] + '\t' + row['rameau_concept_formatted_for_annif'], file=output_train_file)


In [21]:
# Check import with bash: 
! head -n 5 {train_tsv_path}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Une politique pour La Réunion Michel Debré a été élu député de la Réunion en avril 1963, en 1967, en 1968 et en 1973. Ce dernier a pris une part considérable dans une étape essentielle de l'évolution réunionnaise qui est celle de la réalisation effective de la départementalisation entraînant un développement et un progrès considérable dans tous les domaines. Cet ouvrage a pour objectif de déterminer les grandes orientations économiques et sociales de l'île.	https://www.idref.fr/027274292 https://www.idref.fr/027479358
L'Aparole électorale La 4ème de couv. indique : " Sommes-nous la moitié du ciel ou voulons-nous la moitié du gâteau ?… Les élections étaient finies depuis peu en Italie. Elles laissaient derrière elles une déception sensible […], un déchaînement de polémiques acerbes et amères… une fois encore, les femmes étaient mises en accusation… Il ne s’agit pas d’un livre d

In [22]:
# Apply on test dataset
df_test["rameau_concept_formatted_for_annif"] = df_test["rameau_concepts"].apply(lambda x: format_for_annif(x))
df_test["DESCR_cleaned"] = df_test["DESCR"].apply(lambda x: " ".join(x.split()))
df_test[["DESCR_cleaned", "rameau_concept_formatted_for_annif"]].head()

,DESCR_cleaned,rameau_concept_formatted_for_annif
0,"La révolution structurale Mutations ou crises,...",https://www.idref.fr/027249581
1,Esclavage antique et idéologie moderne L'étude...,https://www.idref.fr/027224295 https://www.idr...
2,Cahiers de doléances des femmes en 1789 : et a...,https://www.idref.fr/027225798 https://www.idr...
3,La santé fait ses comptes : une perspective in...,https://www.idref.fr/02736075X https://www.idr...
4,Les étapes de la pensée sociologique : Montesq...,https://www.idref.fr/027301117 https://www.idr...


In [23]:
# Save TSV file for ANNIF
test_tsv_path = os.path.join(annif_data_path, "rameau-test.tsv")
with open(test_tsv_path, 'w', encoding='utf-8') as output_test_file:
        for index, row in df_test.iterrows():
                print(row['DESCR_cleaned'] + '\t' + row['rameau_concept_formatted_for_annif'], file=output_test_file)

### Datasets to get predictions

ANNIF ne sait pas faire la prédiction de chaque notice d'un fichier short-text document (TSV) d'un coup. Il fait une prédiction pour tout le texte du TSV, ce qui n'a pas de sens.

Il faut donc créer un fichier .txt pour chaque notice et lancer la prédiction pour tous les .txt d'un dossier en utilisant annif index

In [24]:
# Train dataset
annif_train_folder_path = os.path.join(annif_data_path, "train/")
for index, row in df_train.iterrows():
    if not os.path.exists(annif_train_folder_path):
        os.makedirs(annif_train_folder_path)
    filename = os.path.join(annif_train_folder_path, str(row['PPN'] + '.txt'))
    with open(filename, 'a') as f:
        f.write(row['DESCR_cleaned'])

In [25]:
# Check number of files in the train folder:
nb_file = !ls {annif_train_folder_path} | wc -l
print(f"Number of files in {annif_train_folder_path} folder: \n{nb_file}")

Number of files in /home/aurelie/ABES/labo-indexation-ai/ANNIF/data/train/ folder: 
['/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)', '115853']


In [26]:
# Test dataset
annif_test_folder_path = os.path.join(annif_data_path, "test/")
for index, row in df_test.iterrows():
    if not os.path.exists(annif_test_folder_path):
        os.makedirs(annif_test_folder_path)
    filename = os.path.join(annif_test_folder_path, str(row['PPN'] + '.txt'))
    with open(filename, 'a') as f:
        f.write(row['DESCR_cleaned'])

In [27]:
# Check number of files in the train folder:
nb_file = !ls {annif_test_folder_path} | wc -l
print(f"Number of files in {annif_test_folder_path} folder: \n{nb_file}")

Number of files in /home/aurelie/ABES/labo-indexation-ai/ANNIF/data/test/ folder: 
['/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)', '27070']


## Test ANNIF

### List all available projects 

In [28]:
# list of projects
project_list = ! annif list-projects
project_list

['/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)',
 'Project ID               Project Name                                 Language  Trained',
 '---------------------------------------------------------------------------------------',
 'rameau-tfidf-snowball-fr TF-IDF French RAMEAU with snowball lemma     fr        False  ',
 'rameau-tfidf-spacy-fr    TF-IDF French RAMEAU with spacy lemma        fr        False  ',
 'rameau-fasttext-fr       FastText French RAMEAU                       fr        None   ',
 'rameau-svc-fr            SVC French RAMEAU                            fr        None   ',
 'rameau-omikuji-parabel-frOmikuji Parabel French                       fr        None   ',
 'rameau-mllm-fr           RAMEAU MLLM project                          fr        False  ',
 'rameau-ensemble-mllm-fr  RAMEAU ensemble with MLLM French             fr        None   ',
 'rameau-ensemble-fasttext-frRAMEAU ensemble with FastTex

### Select project to test

In [35]:
# Select project and parameters
project = "rameau-tfidf-snowball-fr"
njobs = 0
input_file = train_tsv_path
max_nb_concepts = 10
threshold = 0.2 
metric_file_path = os.path.join(annif_report_path, str(project + '.json'))
result_file_path = os.path.join(annif_report_path, str(project + '.csv'))
test_file = test_tsv_path

In [31]:
# Train project
! annif train {project} --jobs {njobs} {input_file}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Backend tfidf: transforming subject corpus
Backend tfidf: creating vectorizer
Backend tfidf: creating similarity index


In [36]:
# Evaluate project
! annif eval --limit {max_nb_concepts} --threshold {threshold} --metrics-file {metric_file_path} --results-file {result_file_path} --jobs {njobs} {project} {test_file}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Writing per subject evaluation results to /home/aurelie/ABES/labo-indexation-ai/ANNIF/reports/rameau-tfidf-snowball-fr.csv
Precision (doc avg):          	0.12882369928638862
Recall (doc avg):             	0.5278674600303408
F1 score (doc avg):           	0.19333392359100954
Precision (subj avg):         	0.024571609443821082
Recall (subj avg):            	0.052584938071034676
F1 score (subj avg):          	0.028848476895531373
Precision (weighted subj avg):	0.20842883496037395
Recall (weighted subj avg):   	0.47828141846485595
F1 score (weighted subj avg): 	0.26290995862717426
Precision (microavg):         	0.1267004972866382
Recall (microavg):            	0.47828141846485595
F1 score (microavg):          	0.20033158672910725
F1@5:                         	0.25437580477122784
NDCG:                         	0.4509288966655731
NDCG@5:                       	0.4170965552330017
ND

In [44]:
# Prediction sur toutes les notices du dossier "test"
suffix = str('_' + project + '.csv')
! annif index -s {suffix} {project} {annif_test_folder_path}

/bin/bash: /home/aurelie/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## Format predictions for future use

In [46]:
csv_files = [f for f in os.listdir(annif_test_folder_path) if f.endswith(".csv")]
print(f"There are {len(csv_files)} files to compile")

There are 27070 files to compile


In [77]:
# Build dataframe
predictions = pd.DataFrame(columns=["PPN", "predictions", "scores"])
for i, file in enumerate(csv_files):
    ppn = file.split('_')[0]
    pred = pd.read_csv(os.path.join(annif_test_folder_path, file), sep='\t', header=None, names=["URI", "pred_concept", "score"])
    predictions.loc[i,"PPN"] = ppn
    predictions.loc[i,"predictions"] = pred["pred_concept"].to_list()
    predictions.loc[i,"scores"] = pred["score"].to_list()

In [93]:
# Show predictions
predictions.head()

,PPN,predictions,scores
0,19598241X,"[Vêtements de femme, Vêtements d'enfant, Robes...","[0.5036637187004089, 0.4903338253498077, 0.435..."
1,143319477,"[Soi (philosophie), Logique antique, Temps (ph...","[0.3134464919567108, 0.3116497695446014, 0.305..."
2,240480805,"[Notaires, Notariat, Actes authentiques, Déont...","[0.5995048880577087, 0.5834233164787292, 0.451..."
3,07908527X,"[Juifs, Judaïsme, Antisémitisme, Relations int...","[0.6071311235427856, 0.5999835729598999, 0.518..."
4,248872214,"[Pensée, Anthropologie, Dans la littérature, É...","[0.4287354648113251, 0.4070886969566345, 0.403..."


In [79]:
# Save dataframe
predictions.to_csv(os.path.join(annif_report_path, str(project + "_predictions.csv")))

In [105]:
data_path

'./../data'

## Merge predictions with existing predictions (including reindexation and indexing if available)

In [ ]:
 # Set files
if save_predictions:
    input_file = "./../data/data_with_reindexation_and_embeddings.csv"
    output_file = "./../data/data_with_reindexation_and_embeddings_withANNIF.csv"
else: 
    input_file = None
    output_file = None

In [113]:
# merge predictions with reindexation file
indexation_file = pd.read_csv(input_file, index_col=0)
print(indexation_file.shape)
indexation_file.head(3)

(99, 17)


,PPN,TITRE,RESUME,RAMEAU,DEWEY,DESCR,presence_chaine_indexation,rameau_chaines_index,rameau_concepts,INDEX_AFE,INDEX_MCR,INDEX_JMF,INDEX_LPL,INDEX_LJZ,INDEX_MPD,embedding_chains,embeddings_autorites
0,000308838,Les sommets de l'État : essai sur l'élite du p...,"u XIXe siècle à nos jours, l'Etat ""fort"" à la ...",Bureaucratie;Classes dirigeantes;Classes dirig...,NaN,Les sommets de l'État : essai sur l'élite du p...,False,"['Bureaucratie', 'Classes dirigeantes', 'Class...","['Bureaucratie', 'Classes dirigeantes', 'Class...",Classes dirigeantes -- France -- Histoire;;Pou...,Pouvoir (sciences sociales) -- Classes dirigea...,Classes dirigeantes -- Relations avec l'État -...,Hauts fonctionnaires -- France;;Classes dirige...,Classes dirigeantes -- France;;Hauts fonctionn...,Classes dirigeantes -- France -- Histoire;;Éli...,"['État', 'Administration publique', 'Fédéralis...","['État', 'Administration publique', 'Fédéralis..."
1,00094758X,Le dollar,"La quatrième de couverture indique : ""Quelle e...",Dollar américain;Finances internationales;Poli...,320,Le dollar La quatrième de couverture indique :...,False,"['Dollar américain', 'Finances internationales...","['Dollar américain', 'Finances internationales...","Dollar américain;;Eurodollar, Marché de l';;Po...",Dollar américain -- Mondialisation;;Dollar amé...,Dollar américain ;;Politique économique -- Éta...,Dollar américain -- Influence -- 20e siècle;;F...,"Dollar américain;;Eurodollar, Marché de l';;Fi...","Dollar américain;;Eurodollar, Marché de l';;Fi...","['Dollar américain', 'Système monétaire intern...","['Dollar américain', 'Système monétaire intern..."
2,003632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",Intellectuels;Intellectuels français,301,Les intellectuels sous la Ve République : 1958...,False,"['Intellectuels', 'Intellectuels français']","['Intellectuels', 'Intellectuels français']",Intellectuels -- France -- 1958-.... (5e Répub...,Intellectuels -- France -- 1945,Intellectuels français -- Sociologie ;;Intelle...,Intellectuels -- France -- 1958 (5e République...,Intellectuels -- France;;Vie intellectuelle --...,Intellectuels -- France -- 1958-.... (5e Répub...,"['Intellectuels -- Activité politique', 'Intel...","['Intellectuels', 'Activité politique', 'Intel..."


In [110]:
# Merge predictions
output = indexation_file.merge(predictions, on="PPN", how="inner")
output.rename(columns={"predictions" : str("predictions_" + project), "scores":str("scores_" + project)}, inplace=True)

In [112]:
output.head(3)

,PPN,TITRE,RESUME,RAMEAU,DEWEY,DESCR,presence_chaine_indexation,rameau_chaines_index,rameau_concepts,INDEX_AFE,INDEX_MCR,INDEX_JMF,INDEX_LPL,INDEX_LJZ,INDEX_MPD,embedding_chains,embeddings_autorites,predictions_rameau-tfidf-snowball-fr,scores_rameau-tfidf-snowball-fr
0,00094758X,Le dollar,"La quatrième de couverture indique : ""Quelle e...",Dollar américain;Finances internationales;Poli...,320,Le dollar La quatrième de couverture indique :...,False,"['Dollar américain', 'Finances internationales...","['Dollar américain', 'Finances internationales...","Dollar américain;;Eurodollar, Marché de l';;Po...",Dollar américain -- Mondialisation;;Dollar amé...,Dollar américain ;;Politique économique -- Éta...,Dollar américain -- Influence -- 20e siècle;;F...,"Dollar américain;;Eurodollar, Marché de l';;Fi...","Dollar américain;;Eurodollar, Marché de l';;Fi...","['Dollar américain', 'Système monétaire intern...","['Dollar américain', 'Système monétaire intern...","[Dollar américain, Système monétaire internati...","[0.5164358615875244, 0.4515779912471771, 0.414..."
1,05224170X,Apprendre à aimer les mathématiques : conditio...,"Les entretiens d'élèves et d'enseignants, anal...",Mathématiques -- Étude et enseignement -- Aspe...,370,Apprendre à aimer les mathématiques : conditio...,True,['Mathématiques -- Étude et enseignement -- As...,"['Mathématiques', 'Étude et enseignement', 'As...",Mathématiques -- Étude et enseignement;;Psycho...,Mathématiques -- Enfants en difficulté d'appre...,Didactique -- Mathématiques;;Apprentissage par...,Apprentissage -- Mathématiques;;Enseignement -...,Mathématiques -- Pédagogie;;Mathématiques -- É...,Mathématiques -- Étude et enseignement -- Fran...,"['Apprentissage par problèmes', ""Psychologie d...","['Apprentissage par problèmes', ""Psychologie d...","[Pédagogie de soutien, Psychologie de l'appren...","[0.4382833838462829, 0.4295188188552856, 0.427..."
2,103220844,"Friedrich A. von Hayek : vie, oeuvres, concepts","La 4e de couverture indique: ""Les œuvres de Fr...",Libéralisme économique,330.157,"Friedrich A. von Hayek : vie, oeuvres, concept...",False,['Libéralisme économique'],['Libéralisme économique'],Économistes -- 20e siècle;;Libéralisme économique,Libéralisme économique;;Économistes -- Prix No...,Libéralisme économique ;;École autrichienne d'...,Économistes -- Libéralisme économique -- Autri...,Keynésianisme;;Libéralisme économique;;Économi...,Keynésianisme;;Libéralisme économique -- 20e s...,"['Keynésianisme', 'Libéralisme (philosophie)',...","['Keynésianisme', 'Libéralisme (philosophie)',...","[Libéralisme (philosophie), Économie de marché...","[0.3712964057922363, 0.3706687390804291, 0.370..."


In [114]:
# Save output
output.to_csv(output_file)